In [ ]:
#Topic Mining

In [1]:
import pandas as pd
import spacy
import time
import nltk

In [46]:
# for now we restrich Restaurants to this number to develop the code
sample_restaurants_to_load = 10000

# Only Arizona Businesses, Change if needed
restaurant_file='processed_data/restaurants_az.csv'
reviews_file   ='processed_data/restaurant_az_reviews.csv'

In [7]:
%%time
# This is the large Spacy English Library
nlp = spacy.load('en_core_web_lg')

Wall time: 15.3 s


In [8]:
# Stopwords for topic mining
stopwords = [line.rstrip('\n') for line in open('config/stopwords.txt', 'r')]

In [29]:
# The words that appear in names of the Restaurants
# Restaurants name may appear multiple time in review, increasing its word frequenty
# For topic mining per restaurant, it is not useful and should be removed
# However words such as 'chicken' when come in restaurant name should be retained
stopnames = [line.rstrip('\n').lower() for line in open('config/names.txt', 'r')]

In [31]:
%%time
# Read Businesses
all_restaurants = pd.read_csv(restaurant_file).drop(labels='Unnamed: 0', axis=1).head(sample_restaurants_to_load)

Wall time: 21.9 ms


In [14]:
%%time
# Read all reviews
all_reviews = pd.read_csv(reviews_file).drop(labels='Unnamed: 0', axis=1)

Wall time: 21 s


In [15]:
%%time
# Retain reviews of selected Businesses
all_reviews = all_reviews[all_reviews.business_id.isin(all_restaurants.business_id)]

Wall time: 537 ms


In [16]:
%%time
# Top 5 Reviews
all_reviews.head()

Wall time: 998 µs


,business_id,name,city,state,business_stars,review_count,categories,review_id,user_id,review_stars,useful,text,date
0,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,Phoenix,AZ,3.5,269,"Chinese, Restaurants",QgV9RPyPUC3cAse1Wxqoow,P3cMpkppvBuVpPD8LBTbBQ,4.0,2,Enjoyed Nee House immensely. No service issues...,2012-04-28 21:08:22
1,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,Phoenix,AZ,3.5,269,"Chinese, Restaurants",1ZTO6zFtVVxtXclHp4TvHQ,b1yLsCdv4ZL_d3INMCZzoA,3.0,0,I'm not sure how I rate this restaurant becaus...,2017-02-09 05:15:25
2,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,Phoenix,AZ,3.5,269,"Chinese, Restaurants",h17ep5S7O8_JMKovooWoVA,TaVuQWmXAhxy_LvIXBs9sg,5.0,0,The food from this place reminds me of home. I...,2016-08-12 21:38:55
3,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,Phoenix,AZ,3.5,269,"Chinese, Restaurants",FVJaiFuf67Dzamax-zq1UQ,aCY0jMl8Jsvx_HQL4D3tmw,4.0,2,Found this place by just driving down the road...,2011-08-24 22:35:28
4,44YFU284Z3KDEy25QyVoUw,Nee House Chinese Restaurant,Phoenix,AZ,3.5,269,"Chinese, Restaurants",zqzOcreb9KBTaESR6qbTSg,TPZbqNXMA2xMXLLd1zL_0A,4.0,2,We eat here on a regular basis. It's like tha...,2013-04-11 02:26:52


In [ ]:
df = (all_reviews.query(' business_id == "'+restaurant['business_id']+'" ')[['name','text']])

In [ ]:
df.name

In [ ]:
all_reviews.query(' business_id == "'+restaurant['business_id']+'" ')[['name','text']]

In [ ]:
total

In [39]:
def clean_name(name):
    name_toks = []
    
    # Nlp doc from Name
    name_doc = nlp(name)
    for token in name_doc:
        
        # Retain Proper nouns in Name
        if token.pos_ == 'PROPN' or token.like_num:
        
            # Lose stop words in Name
            if token.text.lower() not in stopnames:
            
                # All Restaurant name tokens to be remoed from reviews of this reataurant
                name_toks.append(token.text.lower())
    return name_toks

In [40]:
def clean_doc(doc,name_toks):
    
    # Remove punctuation, symbols (#) and stopwords
    #doc = [tok.text for tok in doc if (tok.text.lower() not in stopwords and tok.pos_ != "PUNCT" and tok.pos_ != "SYM")]
    toks = [tok.text for tok in doc if (tok.text.lower() not in stopwords and tok.text.lower() not in name_toks and tok.pos_ != "SYM")]
    
    # Make all tokens lowercase
    doc = [tok.lower() for tok in toks]
    doc = ' '.join(doc).replace("n't",'not').replace(' .','.').replace('  ',' ')
    
    return doc

In [45]:
%%Time
total = len(all_restaurants)
cleansed_text = []
for index, restaurant in all_restaurants.iterrows():
    print(f'Cleaning reviews for restaurant: "{restaurant["name"]:<{40}}" [{index+1:>{5}}/{total:>{5}}]')
    for parsed_review in nlp.pipe(iter(all_reviews.query(' business_id == "'+restaurant['business_id']+'" ')['text']), batch_size=1000, n_threads=8):
        cleansed_text.append(clean_doc(parsed_review,clean_name(restaurant["name"])))

UsageError: Cell magic `%%Timetotal` not found.


In [43]:
all_reviews['cleansed_text'] = cleansed_text

In [44]:
print(all_reviews[['text','cleansed_text']])

                                                   text  \
0     Enjoyed Nee House immensely. No service issues...   
1     I'm not sure how I rate this restaurant becaus...   
2     The food from this place reminds me of home. I...   
3     Found this place by just driving down the road...   
4     We eat here on a regular basis.  It's like tha...   
5     The best Chinese food we've had in a long time...   
6     This 2 star rating is for the service. Althoug...   
7     Service was shotty, but pretty authentic Chine...   
8     My previous review has the real details about ...   
9     This is the perfect neighborhood family run Ch...   
10    After reading many decent reviews and seeing h...   
11    Wow I was very impressed when I first tasted t...   
12    The service to grab a seat is slow like 20 min...   
13    Fresh and authentic Chinese food. We especiall...   
14    This is a nostalgia play - if you want to expe...   
15    This place is way under rated!!! Very close to... 

In [47]:
all_reviews.to_csv('processed_data/reviews.csv')